In [16]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

model = keras.models.load_model('poisonous_plant_classifier_model')

print("done")

done


In [22]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

print("done!")

INFO:tensorflow:Assets written to: /tmp/tmpc3rtc93l/assets


INFO:tensorflow:Assets written to: /tmp/tmpc3rtc93l/assets
2023-03-25 13:36:14.180553: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-25 13:36:14.180610: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-25 13:36:14.180813: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpc3rtc93l
2023-03-25 13:36:14.238653: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-25 13:36:14.238704: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpc3rtc93l
2023-03-25 13:36:14.514304: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-25 13:36:15.083715: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpc3rtc93l
2023-03-25 13:36:15.347590: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

done!


In [12]:
TF_MODEL_FILE_PATH = 'model.tflite' # The default path to the saved TensorFlow Lite model

interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)

In [13]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['input_layer'], 'outputs': ['dense_23']}}

In [14]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

In [18]:
sunflower_path = 'Test_Samples/sunflower_test.jpg'

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(224, 224)
)

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

# for testing tflite
predictions_lite = classify_lite(input_layer=img_array)['dense_23']
score_lite = tf.nn.softmax(predictions_lite)


# for testing model itself
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

class_names = ['Atlantic_Poison_Oak', 'Eastern_Poison_Ivy', 'Not', 'Poison_Sumac']

# testing tflite
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score_lite)], 100 * np.max(score_lite))
)

# testing model itself
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 1s 1s/step
This image most likely belongs to Atlantic_Poison_Oak with a 47.54 percent confidence.
This image most likely belongs to Atlantic_Poison_Oak with a 47.54 percent confidence.
